In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

import os
import shutil
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
import sys
import csv
import pydicom
from skimage.transform import resize
import matplotlib.pyplot as plt
import imageio
    
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve

In [4]:
root_path_to_data = "rsna-pneumonia-detection-challenge"

def make_dirs_for_preprocessed_images(root_path_to_data):
    
        train_images_dir = os.path.join(root_path_to_data, 'stage_2_train_images_preprocessed_hires_jpg')
        test_images_dir = os.path.join(root_path_to_data, 'stage_2_test_images_preprocessed_hires_jpg')
    
        train_images_dir_penum = os.path.join(train_images_dir, 'penum')
        train_images_dir_norm = os.path.join(train_images_dir, 'normal')
    
        test_images_dir_penum = os.path.join(test_images_dir, 'penum')
        test_images_dir_norm = os.path.join(test_images_dir, 'normal')
    
        for dir in [train_images_dir_penum, train_images_dir_norm, test_images_dir_penum, test_images_dir_norm]:
            if not os.path.exists(dir):
                os.makedirs(dir)

# Load the csv as a dict

def load_dicom_as_array(dicom_file):
    dicom_data = pydicom.read_file(dicom_file)
    image_array = dicom_data.pixel_array
    return image_array

def scrub_preprosessed_images(root_path_to_data):
    """
    Delete the content of the directories where the preprocessed images are stored.
    """

    train_images_dir = os.path.join(root_path_to_data, 'stage_2_train_images_preprocessed')
    test_images_dir = os.path.join(root_path_to_data, 'stage_2_test_images_preprocessed')

    train_images_dir_penum = os.path.join(train_images_dir, 'penum')
    train_images_dir_norm = os.path.join(train_images_dir, 'normal')

    test_images_dir_penum = os.path.join(test_images_dir, 'penum')
    test_images_dir_norm = os.path.join(test_images_dir, 'normal')

    for dir in [train_images_dir_penum, train_images_dir_norm, test_images_dir_penum, test_images_dir_norm]:
        for file in os.listdir(dir):
            os.remove(os.path.join(dir, file))

def preprocess_image(image_array, target_size = (150, 150), 
                     normalization_value = None):

    image_array = resize(image_array, target_size, anti_aliasing=True)

    image_array = image_array/np.max(image_array)
    image_array = image_array*255

    if normalization_value is not None:

        image_array = image_array / normalization_value

    image_array = image_array.astype(np.uint8)

    return image_array


def load_csv_as_dict(csv_file):

    data_dict = {}
    with open(csv_file, 'r') as f:
        reader = csv.reader(f)
        header = next(reader)

        for row in reader:
            patient_id = row[0]
            target = row[-1]
            data_dict[patient_id] = target

    return data_dict

def preprocess_dataset(root_path_to_data, scrub_pros = False, use_radiomics = True):

    if scrub_pros:
        scrub_preprosessed_images(root_path_to_data)

    label_file = os.path.join(root_path_to_data, 'stage_2_train_labels.csv')
    label_dict = load_csv_as_dict(label_file)

    # Check with a single image

    # Make new directory for the preprocessed images

    all_training_images = os.listdir(os.path.join(root_path_to_data, 'stage_2_train_images'))

    # Split into training and test sets

    # Here load the split from radiomics

    if use_radiomics:
        f_name_train_test_split = "train_test_split_segmented.csv"
        img_list = load_csv_as_dict(f_name_train_test_split)
        

        train_images_dir = os.path.join(root_path_to_data, 'stage_2_train_images_preprocessed_hires_jpg')
        test_images_dir = os.path.join(root_path_to_data, 'stage_2_test_images_preprocessed_hires_jpg')
    
        train_images_dir_penum = os.path.join(train_images_dir, 'penum')
        train_images_dir_norm = os.path.join(train_images_dir, 'normal')
    
        test_images_dir_penum = os.path.join(test_images_dir, 'penum')
        test_images_dir_norm = os.path.join(test_images_dir, 'normal')

    else:

        print("Error, not yet implemented")

    img_list = load_csv_as_dict(f_name_train_test_split)

    counter = 0

    #sys.exit()

    for key in img_list.keys():

        f_name = key + ".dcm"
        f_name_png = f_name.replace('.dcm', '.jpg')

        img_arr = load_dicom_as_array(os.path.join(root_path_to_data, 'stage_2_train_images', f_name))
        #img_arr = preprocess_image(img_arr, target_size = (256, 256), normalization_value=None)
        img_arr = img_arr/255.0

        #print(np.mean(img_arr))
        #print(np.max(img_arr))

        # Now sort in train, test and normal and pneumonia

        if img_list[key] == 'train':

            label_key = key.split('.')[0] # Hacky AF

            if int(label_dict[label_key]) == 0:
                imageio.imwrite(os.path.join(train_images_dir_norm, f_name_png), img_arr)
            else:
                imageio.imwrite(os.path.join(train_images_dir_penum, f_name_png), img_arr)

        else:
                label_key = key.split('.')[0] # Hacky AF

                if int(label_dict[label_key]) == 0:
                    imageio.imwrite(os.path.join(test_images_dir_norm, f_name_png), img_arr)
                else:
                    imageio.imwrite(os.path.join(test_images_dir_penum, f_name_png), img_arr)

        counter = counter + 1

        print("Preprocessed {} images out of {}, {}".format(
            counter, len(img_list.keys()), counter/len(img_list.keys())), end='\r')

make_dirs_for_preprocessed_images(root_path_to_data)
preprocess_dataset(root_path_to_data, scrub_pros = False)

Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0

Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0

Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0

Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0

Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0

...

Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0

Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0

Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0

Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0

Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0

In [23]:
# Now we have to make a new training and test

train_images_dir = os.path.join(root_path_to_data, 'stage_2_train_images_preprocessed_hires_jpg')
train_images_normal = os.path.join(train_images_dir, "normal")
train_images_pnum = os.path.join(train_images_dir, "penum")

file_list = []

list_of_normal_files = os.listdir(train_images_normal)
list_of_pnum_files = os.listdir(train_images_pnum)

normal_class = np.zeros(len(list_of_normal_files))
pnum_class = np.ones(len(list_of_pnum_files))

list_of_all_files = list_of_normal_files + list_of_pnum_files
all_classes = np.concatenate((normal_class, pnum_class))

print(len(list_of_all_files))
print(len(all_classes))

train_files, validation_files = train_test_split(list_of_all_files, test_size=0.2, stratify=all_classes)

validation_path = os.path.join(root_path_to_data, "stage_2_val_images_preprocessed_hires_jpg")

normal_path = os.path.join(validation_path, "normal")
pnum_path = os.path.join(validation_path, "pnum")

# Move the validation files

counter = 0

for file_name in list_of_all_files:
    if file_name in validation_files:
        file_index = list_of_all_files.index(file_name)
        class_num = all_classes[file_index]
    
        if int(class_num) == 0:
            src = os.path.join(train_images_normal, file_name)
            dest = os.path.join(normal_path, file_name)
            shutil.move(src, dest)
        else:
            src = os.path.join(train_images_pnum, file_name)
            dest = os.path.join(pnum_path, file_name)
            shutil.move(src, dest)
    else:
        
        pass

print(counter)


13518
13518
0
